In [ ]:
# Importar a biblioteca Pandas e declarar o caminho para o arquivo Dados_Candidatos para dar início ao processo de ETL.
import pandas as pd

# Insira o caminho para o arquivo Dados_Candidatos.xlsx. (Ex: D:\Desafio_ISOps\Dados_Candidatos.xlsx)
CAMINHO_PARA_O_ARQUIVO_DADOS_CANDIDATOS = ''

# 1. Leitura e configuração de alguns parâmetros para evitar retornos de falso positivo ao realizar transformações nos dados.

DADOS = pd.read_excel(CAMINHO_PARA_O_ARQUIVO_DADOS_CANDIDATOS)
DADOS.is_copy = False
pd.options.mode.chained_assignment = None

In [ ]:
# 2. Transposição das linhas Etapas em colunas distintas.

dados_transpostos = DADOS.fillna(value=0).pivot_table(index=['ID_Candidato', 'Nome_Candidato', 'ID_Vaga', 'Nome_Vaga', 
                                                            'Data_Candidatura', 'Status', 'Motivo_Reprovação'],
                                                             columns='Etapa',
                                                             values='Data_Movimentacao',
                                                             aggfunc='last').reset_index()

In [ ]:
# 3. Filtrar as datas duplicadas para uma eventual união.

'''
    Criar uma coluna separada com a data de cadastro para selecionar o último valor e aplicar um merge na tabela principal.
'''

dados_cadastro = DADOS[DADOS['Etapa'] == 'Cadastro'][['ID_Candidato', 'Data_Movimentacao']].rename(columns={'Data_Movimentacao': 'Cadastro'})
dados_transpostos = pd.merge(dados_transpostos, dados_cadastro, on='ID_Candidato', how='right')

In [ ]:
# 4. Agregar todos os valores da coluna criada para a coluna principal.

dados_transpostos.loc[~dados_transpostos['Cadastro_x'].isin(dados_transpostos['Cadastro_y']), 'Cadastro_x'] = dados_transpostos['Cadastro_y']

In [ ]:
# 5. Remoção de valores duplicados -> Renomeação da coluna Cadastro_x para seu valor original (Cadastro) -> Remoção de índices extras desnecessários.

dados_transpostos = dados_transpostos.drop_duplicates(subset=['ID_Candidato'], keep='last'
                                                   ).rename(columns={'Cadastro_x':'Cadastro'}
                                                           ).drop(columns=['Cadastro_y']
                                                                 ).reset_index(drop=True)

In [ ]:
# 6. Remapeamento das colunas (suposta ordem cronológica).

colunas_ordenadas = ['ID_Candidato', 'Nome_Candidato', 'ID_Vaga', 'Nome_Vaga', 'Data_Candidatura', 
                     'Status', 'Cadastro', 'Triagem', 'Entrevista TA', 'Teste Técnico', 
                     'Entrevista RH', 'Entrevista Final', 'Oferta', 'Contratação', 'Motivo_Reprovação']

dados_ordenados = dados_transpostos[colunas_ordenadas].reset_index(drop=True)

In [ ]:
'''
7. Substituição dos 0 atribuidos na etapa 2 por valores nulos e 
formatação das colunas de data para atender ao critério estabelecido: Formatação ISO8601
'''

'''
A função verificar_tipo_de_dado_da_celula é usada para verificar se o tipo de dado de dada célula de uma coluna específica
está no formato datetime64[ns] (formato padrão do Excel para células do tipo Data).

Possíveis resultados:
Verdadeiro: O valor correto para atender ao critério de formatação ISO8601 é atribuido à célula.
Falso: Um valor nulo (NA) é atribuido à célula.
'''

def verificar_tipo_de_dado_da_celula(valor_da_celula):
    if pd.api.types.is_datetime64_any_dtype(valor_da_celula):
        return pd.NA if pd.isna(valor_da_celula) else valor_da_celula.date()
    else:
        return valor_da_celula

for coluna in dados_ordenados.columns[0:]:
    dados_ordenados[coluna] = dados_ordenados[coluna].replace(0, pd.NA).apply(verificar_tipo_de_dado_da_celula)

In [ ]:
# 8. Exportar o dataframe final para um arquivo CSV.

dados_ordenados.to_csv('Dados Limpos e Estruturados.csv', index=False, encoding='utf-8-sig', date_format="%Y-%m-%d")